In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/religious-and-philosophical-texts/35895-0.txt
/kaggle/input/religious-and-philosophical-texts/pg2800.txt
/kaggle/input/religious-and-philosophical-texts/pg2680.txt
/kaggle/input/religious-and-philosophical-texts/pg10.txt
/kaggle/input/religious-and-philosophical-texts/pg17.txt
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.REL
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.ALL
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.QRY


In [4]:
!pip install python-terrier==0.12.1

ERROR: Could not find a version that satisfies the requirement python-terrier==0.12.1 (from versions: none)
ERROR: No matching distribution found for python-terrier==0.12.1


In [3]:
import numpy as np
import pandas as pd
import pyterrier as pt

ModuleNotFoundError: No module named 'pyterrier'

In [ ]:
import os
#%%capture
!curl -O https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz
!mv openjdk-11.0.2_linux-x64_bin.tar.gz /usr/lib/jvm/; cd /usr/lib/jvm/; tar -zxvf openjdk-11.0.2_linux-x64_bin.tar.gz
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-11.0.2/bin/java 1
!update-alternatives --set java /usr/lib/jvm/jdk-11.0.2/bin/java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-11.0.2"

# Importing and Indexing Datasets

Datasets used (fields for index are customizable but currently `title` and `abstract` used):
- TREC Covid: `IterDictIndexer` needed but already in the `irds:cord19` directory
- NF Corpus: Need to create own index to run PyTerrier experiments
- MS Marco: `TRECCollectionIndexer` easy to make but gigantic huge corpus or what; index takes forever to make fml and your kaggle might freeze occasionally

## TREC COVID

In [ ]:
trec_covid_dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
trec_covid_corpus = pd.DataFrame(trec_covid_dataset.get_corpus_iter())
# [title, description, narrative] i guess
trec_covid_queries = pd.DataFrame(trec_covid_dataset.get_topics('title'))
trec_covid_qrels = pd.DataFrame(trec_covid_dataset.get_qrels())

In [ ]:
!rm -rf ./cord19-index
trec_covid_indexer = pt.index.IterDictIndexer('./cord19-index')
trec_covid_indexref = trec_covid_indexer.index(trec_covid_dataset.get_corpus_iter(), fields=('title', 'abstract'),  meta={"docno": 20, "title": 256, "abstract": 2048})
trec_covid_index = pt.IndexFactory.of(trec_covid_indexref)

## NF Corpus

In [ ]:
nf_dataset_train = pt.get_dataset("irds:nfcorpus/train")
nf_corpus_train = pd.DataFrame(nf_dataset_train.get_corpus_iter())
# can also be set to 'all' instead of 'title' but not sure how it affects the pyterrier experiments
# # [title, all]
nf_queries_train = pd.DataFrame(nf_dataset_train.get_topics('title'))
nf_qrels_train = pd.DataFrame(nf_dataset_train.get_qrels())

nf_dataset_test = pt.get_dataset("irds:nfcorpus/test")
nf_corpus_test = pd.DataFrame(nf_dataset_test.get_corpus_iter())
# can also be set to 'all' instead of 'title' but not sure how it affects the pyterrier experiments
# # [title, all]
nf_queries_test = pd.DataFrame(nf_dataset_test.get_topics('title'))
nf_qrels_test = pd.DataFrame(nf_dataset_test.get_qrels())

In [ ]:
!rm -rf ./nf_index
!rm -rf ./nf_index2

nf_indexer = pt.IterDictIndexer("./nf_index")
nf_indexref = nf_indexer.index(
    nf_dataset_test.get_corpus_iter(),
    fields=('title', 'abstract'),
    meta={"docno": 20, "title": 256, "abstract": 2048}
)
nf_index = pt.IndexFactory.of(nf_indexref)

nf_indexer2 = pt.IterDictIndexer("./nf_index2")
nf_indexref2 = nf_indexer2.index(nf_dataset_train.get_corpus_iter(), fields=('title', 'abstract'))
nf_index2 = pt.IndexFactory.of(nf_indexref2)

# Running Experiments

In [ ]:
DPH_br = pt.BatchRetrieve(
    trec_covid_index,
    wmodel="DPH",
    metadata=["docno", "title", "abstract"],
    properties={"termpipelines": ""}
)
BM25_br = pt.BatchRetrieve(
    trec_covid_index,
    wmodel="BM25",
    metadata=["docno", "title", "abstract"],
    properties={"termpipelines": ""}
)
# this runs an experiment to obtain results on the TREC COVID queries and qrels
pt.Experiment(
    [DPH_br, BM25_br],
    trec_covid_dataset.get_topics('title'),
    trec_covid_dataset.get_qrels(),
    eval_metrics=["P.5", "P.10", "ndcg_cut.10", "map", "recip_rank"])

In [ ]:
DPH_br = pt.BatchRetrieve(
    nf_index,
    wmodel="DPH",
    metadata=["docno", "title", "abstract"],
    properties={"termpipelines": ""}
)
BM25_br = pt.BatchRetrieve(
    nf_index,
    wmodel="BM25",
    metadata=["docno", "title", "abstract"],
    properties={"termpipelines": ""}
)
# this runs an experiment to obtain results on the TREC COVID queries and qrels
pt.Experiment(
    [DPH_br, BM25_br],
    nf_dataset_test.get_topics('title'),
    nf_dataset_test.get_qrels(),
    eval_metrics=["P.5", "P.10", "ndcg_cut.10", "map", "recip_rank"])

# Query Expansion Techniques

## Part 5: Word Embedding-Based Expansion: Word2vec

In [ ]:
!pip install numpy==1.24.3
!pip install gensim==4.3.1
!pip install scipy==1.10.1

In [ ]:
import pyterrier as pt
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


#from gensim.models import Word2Vec

# Download required NLTK data
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


# Load pre-trained word embedding model
#word2vec_model = api.load("glove-wiki-gigaword-100")
#word2vec_model = KeyedVectors.load_word2vec_format("BioWordVec_PubMed_MIMICIII_d200.vec.bin", binary=True)

dataset_dev = pt.datasets.get_dataset("irds:nfcorpus/dev")

corpus_texts = [
    f"{doc['title']} {doc['abstract']}"
    for doc in dataset_dev.get_corpus_iter()
    if 'title' in doc and 'abstract' in doc
]

from nltk.tokenize import word_tokenize

tokenized_corpus = [
    [w.lower() for w in word_tokenize(text) if w.isalpha()]
    for text in corpus_texts
]
word2vec_model = Word2Vec(
    sentences=tokenized_corpus,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)



stop_words = set(stopwords.words('english'))
topics_df = pd.DataFrame(nf_dataset_test.get_topics('title'))

def expand_query_word2vec(query: str, topn=2):
    tokens = [w.lower() for w in word_tokenize(query) if w.isalpha() and w.lower() not in stop_words]
    expanded = []
    for term in tokens:
        expanded.append(term)  # original term weighted more
        try:
            for w, sim in word2vec_model.wv.most_similar(term, topn=topn):
                if sim > 0.65:
                    expanded.append(w)
        except KeyError:
            continue
    return " ".join(expanded)


expanded_queries_w2v = []
for _, row in topics_df.iterrows():
    qid = row['qid']
    query = row['query']
    exp_query = expand_query_word2vec(query)
    expanded_queries_w2v.append({'qid': qid, 'query': exp_query})
expanded_queries_df = pd.DataFrame(expanded_queries_w2v)

expanded_queries_w2v_df = pd.DataFrame(expanded_queries_w2v)



#### experiment

In [ ]:
import re

BM25_br = pt.BatchRetrieve(
    nf_index,
    wmodel="BM25",
    metadata=["docno", "title", "abstract"],
    properties={"termpipelines": ""}
)
BM25_br = pt.terrier.Retriever(nf_index, wmodel="BM25") % 100


def escape_pyterrier_query(query: str) -> str:
    """
    Escapes and sanitizes a query string to prevent PyTerrier parsing errors.
    """
    # Remove markdown-like formatting (e.g., **bold**, *italic*)
    query = re.sub(r'\*+', '', query)  # Remove asterisks

    # Remove other non-alphanumeric characters that could cause issues
    query = re.sub(r'[\':"{}<>|\[\]]', ' ', query)  # Remove problematic symbols including brackets

    # Replace multiple consecutive spaces with a single space
    query = re.sub(r'\s+', ' ', query).strip()

    # Remove the word "TITLE" (case insensitive)
    #query = re.sub(r'\bTITLE\b', '', query, flags=re.IGNORECASE)

    # Remove the word "TITLE" (case insensitive)
    #query = re.sub(r'\bSEP\b', '', query, flags=re.IGNORECASE)

    #print(query)
    #print("####")

    return query


def escape_pyterrier_queries(queries):
    """
    Apply the escaping function to a list of queries.
    """
    #for obj in queries:

    #    return []
    res = []

    #for query in queries:
    #    print({'qid': query['qid'], 'query': escape_pyterrier_query(query['query'])})

    return [{'qid': query['qid'], 'query': escape_pyterrier_query(query['query'])} for query in queries]


pt.Experiment(
    [DPH_br, BM25_br],
    pd.DataFrame(escape_pyterrier_queries(expanded_queries_w2v)),
    nf_dataset_test.get_qrels(),
    eval_metrics=["P.5", "P.10", "ndcg_cut.10", "map", "recip_rank"])






## Part 6: Pseudo-Relevance Feedback (PRF) Using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

def clean_tokens(text):
    return set(
        w.lower() for w in word_tokenize(text)
        if w.isalpha() and w.lower() not in stop_words
    )


# === PRF-BASED QUERY EXPANSION FUNCTION ===
def expand_query_prf(query: str, index, bm25, k=5):
    """
    Performs PRF expansion using top-k BM25 documents + TF-IDF similarity.
    """

    res = bm25.search(query).head(k)
    medical_docs = (res["title"] + " " + res["abstract"]).tolist()

    if not medical_docs:
        return query


    vectorizer = TfidfVectorizer(stop_words='english')
    doc_vectors = vectorizer.fit_transform(medical_docs)
    query_vector = vectorizer.transform([query])

    cosine_similarities = cosine_similarity(query_vector, doc_vectors).flatten()
    top_doc_index = cosine_similarities.argmax()
    top_doc = medical_docs[top_doc_index]


    top_doc_tokens = clean_tokens(top_doc)
    query_tokens = clean_tokens(query)
    expansion_terms = top_doc_tokens.difference(query_tokens)



    expansion_terms = list(expansion_terms)[:5]

    expanded_query = query + " " + " ".join(expansion_terms)


    return expanded_query


In [ ]:

BM25_br = pt.BatchRetrieve(
    nf_index,
    wmodel="BM25",
    metadata=["docno", "title", "abstract"],
    properties={"termpipelines": ""}
)

def prf_expand_df(df):
    return pd.DataFrame([
        {
            "qid": row["qid"],
            "query": expand_query_prf(row["query"], index=nf_index, bm25=BM25_br)
        }
        for _, row in df.iterrows()
    ])


def prf_wrapper(query_obj):
    query = query_obj["query"]
    qid = query_obj["qid"]
    new_query = expand_query_prf(query, index=nf_index, bm25=BM25_br)
    return pd.DataFrame([{"qid": qid, "query": new_query}])

prf_pipe = pt.apply.generic(prf_expand_df) >> BM25_br


In [ ]:
topics_df = pd.DataFrame(nf_dataset_test.get_topics('title'))

print(topics_df.head(5))
print(BM25_br.search(topics_df.iloc[0]["query"]))


In [ ]:
topics_df = pd.DataFrame(nf_dataset_test.get_topics('title'))

pt.Experiment(
    [BM25_br, prf_pipe],
    topics_df,
    nf_dataset_test.get_qrels(),
    eval_metrics=["P.5", "P.10", "ndcg_cut.10", "map", "recip_rank"])
